Deep Learning
=============

任务 3 Assignment 3
------------

接着 `2_fullyconnected.ipynb` ，你训练一个逻辑回归和神经网络模型

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

这是人物的目标是探索正则化技巧

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

数据来自`1_notmnist.ipynb` 

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


重新格式化为一个更适合我们要训练的模型的形状

Reformat into a shape that's more adapted to the models we're going to train:
- 数据是一个矩阵
- 标签是一个独热编码
- data as a flat matrix,
- labels as float one-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# 计算精准度
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
问题 1
---------

从逻辑和神经网络模型来介绍和调整L2正则化。记住L2总计是在损失的标准权重上增加一个惩罚。在TensorFlow里，可以使用`nn.l2_loss（t）`来计算张量`t`的L2损失。要正确计算正则化应该改善你的训练和测试的精准度

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [42]:
batch_size = 128  
regularation_param = 0.0001  
  
graph = tf.Graph()  
with graph.as_default():
    # -----------------------------------------1
    # 输入 
    # placeholder 插入一个待初始化的张量占位符
    # 重要事项：这个张量被求值时会产生错误。 
    # 它的值必须在Session.run(), Tensor.eval() 或 Operation.run() 中使用feed_dict的这个可选参数来填充。
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # 创建一个常量张量
    # tf_valid_dataset = Tensor("Const:0", shape=(10000, 784), dtype=float32)
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # ------------------------------------------2
    # 变量
    # 当你训练一个模型的时候，你使用变量去保存和更新参数。
    # 在Tensorflow中变量是内存缓冲区中保存的张量（tensor）。
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # -------------------------------------------3
    # 训练计算.
    train_logits = tf.add(tf.matmul(tf_train_dataset, weights), biases)
    
    print(tf_train_dataset)
    print(weights)
    print(biases)
    print(train_logits)
    
    #print(tf_valid_dataset.shape)
    #print(weights.shape)
    
    valid_logits = tf.add(tf.matmul(tf_valid_dataset, weights), biases)
    test_logits = tf.add(tf.matmul(tf_test_dataset, weights), biases)
    # reduce_mean可跨越维度的计算张量各元素的平均值
    # 计算loss是代价值，也就是我们要最小化的值
    # 第一个参数logits：就是神经网络最后一层的输出，
    # 如果有batch的话，它的大小就是[batchsize，num_classes]，单样本的话，大小就是num_classes
    # 第二个参数labels：实际的标签，大小同上
    # 第一步是先对网络最后一层的输出做一个softmax，这一步通常是求取输出属于某一类的概率
    # 第二步是softmax的输出向量[Y1，Y2,Y3...]和样本的实际标签做一个交叉熵
    # 注意！！！这个函数的返回值并不是一个数，而是一个向量.
    # 如果要求交叉熵，我们要再做一步tf.reduce_sum操作,就是对向量里面所有元素求和，最后才得到. 
    # 如果求loss，则要做一步tf.reduce_mean操作，对向量求均值！
    
    # 如果直接将l2_loss加到train_loss上，每次的train_loss都特别大，几乎只取决于l2_loss
    # 为了让原本的train_loss与l2_loss都能较好地对参数调整方向起作用，它们应当至少在同一个量级
    # 这里还有一个重要的点，Hyper Parameter: β
    # 我觉得这是一个拍脑袋参数，取什么值都行，但效果会不同，我这里解释一下我取β=0.001的理由
    # 为了让原本的train_loss与l2_loss都能较好地对参数调整方向起作用，它们应当至少在同一个量级
    # 观察不加l2_loss，step 0 时，train_loss在300左右
    # 加l2_loss后， step 0 时，train_loss在300000左右
    # 因此给l2_loss乘0.0001使之降到同一个量级
    # 原始 loss=18
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=train_logits, labels=tf_train_labels))
    # 原始L2后loss=58000
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits)) * tf.nn.l2_loss(weights)  
    # 使用Hyper Parameter 0.0001 相乘来降低L2的影响，使用后loss=16
    hpl2 = regularation_param * tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits))
    # print(loss)
    loss = tf.add(loss, hpl2)

    
    #loss = tf.add(loss,
    # -------------------------------------------4
    # 最优化.因为深度学习常见的是对于梯度的优化，也就是说，优化器最后其实就是各种对于梯度下降算法的优化。 
    # 函数training()通过梯度下降法为最小化损失函数增加了相关的优化操作，在训练过程中，
    # 先实例化一个优化函数，比如 tf.train.GradientDescentOptimizer，并基于一定的学习率进行梯度优化训练：
    # learning_rate参数：要使用的学习率 
    # minimize：非常常用的一个函数 通过更新var_list来减小loss，这个函数就是前面compute_gradients() 和apply_gradients().的结合
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    #--------------------------------------------5
    # 预测训练, 验证,和测试数据.
    # tf.nn.softmax仅产生将softmax function应用于输入张量的结果。 
    # softmax 压扁输入，使sum(输入)= 1;这是一种正常化的方式。 
    # softmax的输出形状与输入相同 – 它只是对值进行归一化。 
    # softmax的输出可以解释为概率。
    # a = tf.constant(np.array([[.1, .3, .5, .9]]))
    # print s.run(tf.nn.softmax(a))
    # [[ 0.16838508  0.205666    0.25120102  0.37474789]]
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

Tensor("Mean:0", shape=(), dtype=float32)


1. 使用梯度计算train_loss，用tf.Graph()创建一个计算单元
2. 用tf.constant将dataset和label转为tensorflow可用的训练格式（训练中不可修改）
3. 用tf.truncated_normal生成正太分布的数据，作为W的初始值，初始化b为可变的0矩阵
4. 用tf.variable将上面的矩阵转为tensorflow可用的训练格式（训练中可以修改）
5. 用tf.matmul实现矩阵相乘，计算WX+b，这里实际上logit只是一个变量，而非结果
6. 用tf.nn.softmax_cross_entropy_with_logits计算WX+b的结果相较于原来的label的train_loss，并求均值
7. 使用梯度找到最小train_loss

In [32]:
num_steps = 3001  
  
with tf.Session(graph=graph) as session:  
    tf.initialize_all_variables().run()  
    print("Initialized")  
    for step in range(num_steps):  
        # 在训练数据中选择一个已被随机化的偏移量.
        # 提醒: 我们能使用更好的随机化穿过所有数据.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)  
        # 生成一个小批量数据
        batch_data = train_dataset[offset:(offset + batch_size), :]  
        batch_labels = train_labels[offset:(offset + batch_size), :]  
        # feed_dict的作用是给使用placeholder创建出来的tensor赋值。
        # 其实，他的作用更加广泛：feed 使用一个 值临时替换一个 op 的输出结果. 
        # 你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失.
        #  传递值到tf的命名空间  
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}  
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)  
        if (step % 500 == 0):  
            print("Minibatch loss at step %d: %f" % (step, l))  
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))  
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))  
    print("Test accuracy: %.1f%% with regularation_param = %f " %( accuracy(test_prediction.eval(), test_labels), regularation_param)) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 16.740925
Minibatch accuracy: 8.6%
Validation accuracy: 14.7%
Minibatch loss at step 500: 1.502702
Minibatch accuracy: 81.2%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.583782
Minibatch accuracy: 79.7%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.957507
Minibatch accuracy: 80.5%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 0.795472
Minibatch accuracy: 85.9%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 1.022002
Minibatch accuracy: 78.1%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 0.978406
Minibatch accuracy: 78.9%
Validation accuracy: 79.7%
Test accuracy: 86.8% with regularation_param = 0.000100 


---
问题 2
---------
让我们展示一个计算过拟合的情况。把你的训练数据限制在很少的数量。看看发生什么

Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [34]:
num_steps = 3001  
  
with tf.Session(graph=graph) as session:  
    tf.initialize_all_variables().run()  
    print("Initialized")  
    for step in range(num_steps):  
        # 在训练数据中选择一个已被随机化的偏移量.
        # 提醒: 我们能使用更好的随机化穿过所有数据.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)  
        if(offset > 10000):
            offset = 5000
        # 生成一个小批量数据
        batch_data = train_dataset[offset:(offset + batch_size), :]  
        batch_labels = train_labels[offset:(offset + batch_size), :]  
        # feed_dict的作用是给使用placeholder创建出来的tensor赋值。
        # 其实，他的作用更加广泛：feed 使用一个 值临时替换一个 op 的输出结果. 
        # 你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失.
        #  传递值到tf的命名空间  
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}  
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)  
        if (step % 500 == 0):  
            print("Minibatch loss at step %d: %f" % (step, l))  
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))  
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))  
    print("Test accuracy: %.1f%% with regularation_param = %f " %( accuracy(test_prediction.eval(), test_labels), regularation_param)) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 15.266306
Minibatch accuracy: 21.9%
Validation accuracy: 17.0%
Minibatch loss at step 500: 0.285131
Minibatch accuracy: 100.0%
Validation accuracy: 67.6%
Minibatch loss at step 1000: 0.269983
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 1500: 0.256638
Minibatch accuracy: 100.0%
Validation accuracy: 68.2%
Minibatch loss at step 2000: 0.238596
Minibatch accuracy: 100.0%
Validation accuracy: 72.2%
Minibatch loss at step 2500: 0.226437
Minibatch accuracy: 100.0%
Validation accuracy: 72.1%
Minibatch loss at step 3000: 0.215375
Minibatch accuracy: 100.0%
Validation accuracy: 72.0%
Test accuracy: 78.9% with regularation_param = 0.000100 


---
问题 3
---------

在神经网络的隐藏层引入Dropout。记住Dropout应该只有在训练期间引入，不是评估期间，否则你的评估结果将会随机。TensorFlow提供`nn.dropout()`，但是你要确保只有它在训练期间插入。

Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.



What happens to our extreme overfitting case?

---

In [47]:
batch_size = 128  
hidden_nodes = 1024
regularation_param = 0.0001  
keep_prob = 0.8

graph = tf.Graph()  
with graph.as_default():
    # -----------------------------------------1
    # 输入 
    # placeholder 插入一个待初始化的张量占位符
    # 重要事项：这个张量被求值时会产生错误。 
    # 它的值必须在Session.run(), Tensor.eval() 或 Operation.run() 中使用feed_dict的这个可选参数来填充。
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # 创建一个常量张量
    # tf_valid_dataset = Tensor("Const:0", shape=(10000, 784), dtype=float32)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # ------------------------------------------2
    # 变量
    
    # 当你训练一个模型的时候，你使用变量去保存和更新参数。
    # 在Tensorflow中变量是内存缓冲区中保存的张量（tensor）。

    # 第一层
    # truncated_normal 从一个正态分布片段中输出随机数值,
    # 生成的值会遵循一个指定了平均值和标准差的正态分布，只保留两个标准差以内的值，超出的值会被弃掉重新生成。
    # 返回 一个指定形状并用正态分布片段的随机值填充的张量
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    # 第二层
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # -------------------------------------------3
    # 训练计算.
    # logits = tf.matmul(tf_train_dataset, weights) + biases
    
    drop = tf.nn.dropout(tf_train_dataset, keep_prob=keep_prob)
    
    train_logits = tf.add(tf.matmul(drop, weights1),biases1)
    train_logits = tf.nn.relu(train_logits)
    train_logits = tf.add(tf.matmul(train_logits,weights2),biases2)

    valid_logits = tf.add(tf.matmul(tf_valid_dataset, weights1),biases1)
    valid_logits = tf.nn.relu(valid_logits)
    valid_logits = tf.add(tf.matmul(valid_logits,weights2),biases2)

    test_logits = tf.add(tf.matmul(tf_test_dataset, weights1),biases1)
    test_logits = tf.nn.relu(test_logits)
    test_logits = tf.add(tf.matmul(test_logits,weights2),biases2)
    # reduce_mean可跨越维度的计算张量各元素的平均值
    # 计算loss是代价值，也就是我们要最小化的值
    # 第一个参数logits：就是神经网络最后一层的输出，
    # 如果有batch的话，它的大小就是[batchsize，num_classes]，单样本的话，大小就是num_classes
    # 第二个参数labels：实际的标签，大小同上
    # 第一步是先对网络最后一层的输出做一个softmax，这一步通常是求取输出属于某一类的概率
    # 第二步是softmax的输出向量[Y1，Y2,Y3...]和样本的实际标签做一个交叉熵
    # 注意！！！这个函数的返回值并不是一个数，而是一个向量.
    # 如果要求交叉熵，我们要再做一步tf.reduce_sum操作,就是对向量里面所有元素求和，最后才得到. 
    # 如果求loss，则要做一步tf.reduce_mean操作，对向量求均值！
    
    # 如果直接将l2_loss加到train_loss上，每次的train_loss都特别大，几乎只取决于l2_loss
    # 为了让原本的train_loss与l2_loss都能较好地对参数调整方向起作用，它们应当至少在同一个量级
    # 这里还有一个重要的点，Hyper Parameter: β
    # 我觉得这是一个拍脑袋参数，取什么值都行，但效果会不同，我这里解释一下我取β=0.001的理由
    # 为了让原本的train_loss与l2_loss都能较好地对参数调整方向起作用，它们应当至少在同一个量级
    # 观察不加l2_loss，step 0 时，train_loss在300左右
    # 加l2_loss后， step 0 时，train_loss在300000左右
    # 因此给l2_loss乘0.0001使之降到同一个量级
    # 原始 loss=18
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=train_logits, labels=tf_train_labels))
    # 原始L2后loss=58000
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits)) * tf.nn.l2_loss(weights)  
    # 使用Hyper Parameter 0.0001 相乘来降低L2的影响，使用后loss=16
    #hpl2 = regularation_param * tf.nn.l2_loss(weights)
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits))
    #loss = tf.add(loss, hpl2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits)) + regularation_param * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))  

    
    #loss = tf.add(loss,
    # -------------------------------------------4
    # 最优化.因为深度学习常见的是对于梯度的优化，也就是说，优化器最后其实就是各种对于梯度下降算法的优化。 
    # 函数training()通过梯度下降法为最小化损失函数增加了相关的优化操作，在训练过程中，
    # 先实例化一个优化函数，比如 tf.train.GradientDescentOptimizer，并基于一定的学习率进行梯度优化训练：
    # learning_rate参数：要使用的学习率 
    # minimize：非常常用的一个函数 通过更新var_list来减小loss，这个函数就是前面compute_gradients() 和apply_gradients().的结合
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    #--------------------------------------------5
    # 预测训练, 验证,和测试数据.
    # tf.nn.softmax仅产生将softmax function应用于输入张量的结果。 
    # softmax 压扁输入，使sum(输入)= 1;这是一种正常化的方式。 
    # softmax的输出形状与输入相同 – 它只是对值进行归一化。 
    # softmax的输出可以解释为概率。
    # a = tf.constant(np.array([[.1, .3, .5, .9]]))
    # print s.run(tf.nn.softmax(a))
    # [[ 0.16838508  0.205666    0.25120102  0.37474789]]
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

In [48]:
num_steps = 3001  
  
with tf.Session(graph=graph) as session:  
    tf.initialize_all_variables().run()  
    print("Initialized")  
    for step in range(num_steps):  
        # 在训练数据中选择一个已被随机化的偏移量.
        # 提醒: 我们能使用更好的随机化穿过所有数据.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)  
        # 生成一个小批量数据
        batch_data = train_dataset[offset:(offset + batch_size), :]  
        batch_labels = train_labels[offset:(offset + batch_size), :]  
        # feed_dict的作用是给使用placeholder创建出来的tensor赋值。
        # 其实，他的作用更加广泛：feed 使用一个 值临时替换一个 op 的输出结果. 
        # 你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失.
        #  传递值到tf的命名空间  
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}  
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)  
        if (step % 500 == 0):  
            print("Minibatch loss at step %d: %f" % (step, l))  
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))  
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))  
    print("Test accuracy: %.1f%% with regularation_param = %f " %( accuracy(test_prediction.eval(), test_labels), regularation_param)) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 425.041931
Minibatch accuracy: 9.4%
Validation accuracy: 26.6%
Minibatch loss at step 500: 69.365395
Minibatch accuracy: 77.3%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 40.421841
Minibatch accuracy: 75.8%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 33.481506
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 31.675150
Minibatch accuracy: 75.8%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 31.951332
Minibatch accuracy: 75.8%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 24.339428
Minibatch accuracy: 75.8%
Validation accuracy: 80.7%
Test accuracy: 88.7% with regularation_param = 0.000100 


---
问题 4
---------

尝试获得表现，你能使用多层模型！使用深层网络后测试精准度最好的报告是是97.1

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

你能探索一个方式是增加多层

One avenue you can explore is to add multiple layers.

另一个方法是使用学习速率衰减

迭代的次数越多需要的学习速率越应该小，这样才会使loss的值逐渐降低。
否则会产生在一小范围内loss的值来回震荡的现象。

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [63]:
batch_size = 128
# 隐藏节点
hidden_nodes = 1024
# L2缩放率
regularation_param = 0.0001  

graph = tf.Graph()  
def compute_logits(data, weightss, biasess, dropout_vals=None):  
    temp = data  
    if dropout_vals:  
        for w,b,d in zip(weightss[:-1], biasess[:-1], dropout_vals[:-1]):  
            temp = tf.nn.relu_layer(tf.nn.dropout(temp, d), w, b)  
        temp = tf.matmul(temp, weightss[-1]) + biasess[-1]  
    else:  
        for w,b in zip(weightss[:-1], biasess[:-1]):  
            temp = tf.nn.relu_layer(temp, w, b)  
        temp = tf.matmul(temp, weightss[-1]) + biasess[-1]  
    return temp  
with graph.as_default():
    # -----------------------------------------1
    # 输入 
    # placeholder 插入一个待初始化的张量占位符
    # 重要事项：这个张量被求值时会产生错误。 
    # 它的值必须在Session.run(), Tensor.eval() 或 Operation.run() 中使用feed_dict的这个可选参数来填充。
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # 创建一个常量张量
    # tf_valid_dataset = Tensor("Const:0", shape=(10000, 784), dtype=float32)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # ------------------------------------------2
    # 变量
    # #该函数返回以下结果
    # decayed_learning_rate = learning_rate *
    # decay_rate ^ (global_step / decay_steps)
    # 初始的学习速率
    starter_learning_rate = 0.1 
    # 全局的step，与 decay_step 和 decay_rate一起决定了 learning rate的变化
    global_step = tf.Variable(0, trainable=False)
    # 衰减速度
    decay_steps = 100
    # 衰减系数
    decay_rate = 0.5
    # 如果staircase=True，那就表明每decay_steps次计算学习速率变化，更新原始学习速率.
    # 如果是False，那就是每一步都更新学习速率
    staircase = True
    # 指数衰减:法通过这个函数，可以先使用较大的学习率来快速得到一个比较优的解，然后随着迭代的继续逐步减小学习率，使得模型在训练后期更加稳定
    # 87.7% 仅仅指数衰减
    tf.train.exponential_decay(starter_learning_rate,global_step,decay_steps,decay_rate,staircase)
    
    # 当你训练一个模型的时候，你使用变量去保存和更新参数。
    # 在Tensorflow中变量是内存缓冲区中保存的张量（tensor）。

    # 第一层
    # truncated_normal 从一个正态分布片段中输出随机数值,
    # 生成的值会遵循一个指定了平均值和标准差的正态分布，只保留两个标准差以内的值，超出的值会被弃掉重新生成。
    # 返回 一个指定形状并用正态分布片段的随机值填充的张量
    # 数字平方根
    x = 2.0
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev = np.sqrt(x / hidden_nodes)))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    # 第二层
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, int(hidden_nodes / 2)], stddev = np.sqrt(x / hidden_nodes / 2)))
    biases2 = tf.Variable(tf.zeros([hidden_nodes / 2]))
    hidden_nodes = int(hidden_nodes / 2)
    
    # 第三层
    weights3 = tf.Variable(tf.truncated_normal([hidden_nodes, int(hidden_nodes / 2)], stddev = np.sqrt(x / hidden_nodes / 2)))
    biases3 = tf.Variable(tf.zeros([hidden_nodes / 2]))
    hidden_nodes = int(hidden_nodes / 2)
    
    # 第四层 94.5
    weights4 = tf.Variable(tf.truncated_normal([hidden_nodes, int(hidden_nodes / 2)], stddev = np.sqrt(x / hidden_nodes / 2)))
    biases4 = tf.Variable(tf.zeros([hidden_nodes / 2]))
    hidden_nodes = int(hidden_nodes / 2)
    
    # 第五层 94.5
    weights5 = tf.Variable(tf.truncated_normal([hidden_nodes, int(hidden_nodes / 2)], stddev = np.sqrt(x / hidden_nodes / 2)))
    biases5 = tf.Variable(tf.zeros([hidden_nodes / 2]))
    hidden_nodes = int(hidden_nodes / 2)
    
    # 最后一层
    weights_end = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev = np.sqrt(x / num_labels)))
    biases_end  = tf.Variable(tf.zeros([num_labels]))
    
    # -------------------------------------------3
    # 训练计算.
    # logits = tf.matmul(tf_train_dataset, weights) + biases
    # DropDown的保留概率
    keep_prob = 0.8
    drop = tf.nn.dropout(tf_train_dataset, keep_prob=keep_prob)
    
    train_logits = compute_logits(tf_train_dataset, [weights1, weights2, weights3,weights4,weights5, weights_end], 
                                  [biases1,biases2,biases3,biases4,biases5, biases_end],  
                            dropout_vals=(1.0,0.95,0.95,0.95,0.95,1.0))
    
    # train_logits = tf.add(tf.matmul(drop, weights1),biases1)
    # train_logits = tf.nn.relu(train_logits)
    # train_logits = tf.add(tf.matmul(train_logits,weights2),biases2)

    valid_logits = compute_logits(tf_valid_dataset, [weights1, weights2, weights3,weights4,weights5, weights_end], 
                              [biases1,biases2,biases3,biases4,biases5, biases_end])
    
    # valid_logits = tf.add(tf.matmul(tf_valid_dataset, weights1),biases1)
    # valid_logits = tf.nn.relu(valid_logits)
    # valid_logits = tf.add(tf.matmul(valid_logits,weights2),biases2)

    test_logits = compute_logits(tf_test_dataset, [weights1, weights2, weights3,weights4,weights5, weights_end], 
                              [biases1,biases2,biases3,biases4,biases5, biases_end])
    
    # test_logits = tf.add(tf.matmul(tf_test_dataset, weights1),biases1)
    # test_logits = tf.nn.relu(test_logits)
    # test_logits = tf.add(tf.matmul(test_logits,weights2),biases2)
    
    # reduce_mean可跨越维度的计算张量各元素的平均值
    # 计算loss是代价值，也就是我们要最小化的值
    # 第一个参数logits：就是神经网络最后一层的输出，
    # 如果有batch的话，它的大小就是[batchsize，num_classes]，单样本的话，大小就是num_classes
    # 第二个参数labels：实际的标签，大小同上
    # 第一步是先对网络最后一层的输出做一个softmax，这一步通常是求取输出属于某一类的概率
    # 第二步是softmax的输出向量[Y1，Y2,Y3...]和样本的实际标签做一个交叉熵
    # 注意！！！这个函数的返回值并不是一个数，而是一个向量.
    # 如果要求交叉熵，我们要再做一步tf.reduce_sum操作,就是对向量里面所有元素求和，最后才得到. 
    # 如果求loss，则要做一步tf.reduce_mean操作，对向量求均值！
    
    # 如果直接将l2_loss加到train_loss上，每次的train_loss都特别大，几乎只取决于l2_loss
    # 为了让原本的train_loss与l2_loss都能较好地对参数调整方向起作用，它们应当至少在同一个量级
    # 这里还有一个重要的点，Hyper Parameter: β
    # 我觉得这是一个拍脑袋参数，取什么值都行，但效果会不同，我这里解释一下我取β=0.001的理由
    # 为了让原本的train_loss与l2_loss都能较好地对参数调整方向起作用，它们应当至少在同一个量级
    # 观察不加l2_loss，step 0 时，train_loss在300左右
    # 加l2_loss后， step 0 时，train_loss在300000左右
    # 因此给l2_loss乘0.0001使之降到同一个量级
    # 原始 loss=18
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=train_logits, labels=tf_train_labels))
    # 原始L2后loss=58000
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits)) * tf.nn.l2_loss(weights)  
    # 使用Hyper Parameter 0.0001 相乘来降低L2的影响，使用后loss=16
    #hpl2 = regularation_param * tf.nn.l2_loss(weights)
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits))
    #loss = tf.add(loss, hpl2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits)) + regularation_param * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))  

    
    #loss = tf.add(loss,
    # -------------------------------------------4
    # 最优化.因为深度学习常见的是对于梯度的优化，也就是说，优化器最后其实就是各种对于梯度下降算法的优化。 
    # 函数training()通过梯度下降法为最小化损失函数增加了相关的优化操作，在训练过程中，
    # 先实例化一个优化函数，比如 tf.train.GradientDescentOptimizer，并基于一定的学习率进行梯度优化训练：
    # learning_rate参数：要使用的学习率 
    # minimize：非常常用的一个函数 通过更新var_list来减小loss，这个函数就是前面compute_gradients() 和apply_gradients().的结合
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss, global_step=global_step)
    
    
    #--------------------------------------------5
    # 预测训练, 验证,和测试数据.
    # tf.nn.softmax仅产生将softmax function应用于输入张量的结果。 
    # softmax 压扁输入，使sum(输入)= 1;这是一种正常化的方式。 
    # softmax的输出形状与输入相同 – 它只是对值进行归一化。 
    # softmax的输出可以解释为概率。
    # a = tf.constant(np.array([[.1, .3, .5, .9]]))
    # print s.run(tf.nn.softmax(a))
    # [[ 0.16838508  0.205666    0.25120102  0.37474789]]
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

In [65]:
num_steps = 10000  
  
with tf.Session(graph=graph) as session:  
    tf.initialize_all_variables().run()  
    print("Initialized")  
    for step in range(num_steps):  
        # 在训练数据中选择一个已被随机化的偏移量.
        # 提醒: 我们能使用更好的随机化穿过所有数据.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)  
        # 生成一个小批量数据
        batch_data = train_dataset[offset:(offset + batch_size), :]  
        batch_labels = train_labels[offset:(offset + batch_size), :]  
        # feed_dict的作用是给使用placeholder创建出来的tensor赋值。
        # 其实，他的作用更加广泛：feed 使用一个 值临时替换一个 op 的输出结果. 
        # 你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失.
        #  传递值到tf的命名空间  
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}  
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)  
        if (step % 500 == 0):  
            print("Minibatch loss at step %d: %f" % (step, l))  
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))  
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))  
    print("Test accuracy: %.1f%% with regularation_param = %f " %( accuracy(test_prediction.eval(), test_labels), regularation_param)) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.387564
Minibatch accuracy: 10.9%
Validation accuracy: 18.6%
Minibatch loss at step 500: 0.471650
Minibatch accuracy: 87.5%
Validation accuracy: 85.1%
Minibatch loss at step 1000: 0.577436
Minibatch accuracy: 82.0%
Validation accuracy: 86.4%
Minibatch loss at step 1500: 0.347609
Minibatch accuracy: 92.2%
Validation accuracy: 87.3%
Minibatch loss at step 2000: 0.349841
Minibatch accuracy: 93.0%
Validation accuracy: 87.8%
Minibatch loss at step 2500: 0.397478
Minibatch accuracy: 90.6%
Validation accuracy: 88.0%
Minibatch loss at step 3000: 0.475457
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 3500: 0.454240
Minibatch accuracy: 88.3%
Validation accuracy: 89.0%
Minibatch loss at step 4000: 0.361092
Minibatch accuracy: 91.4%
Validation accuracy: 88.8%
Minibatch loss at step 4500: 0.352299
Minibatch accuracy: 89.8%
Validation accuracy: 89.2%
Minibatc